<a href="https://colab.research.google.com/github/nedokormysh/Text_Gen_RNN/blob/model/CharRNN%2BWordRNN_Infinite_Joke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-Level LSTM
Обучим модель на тексте книги "Бесконечная шутка", после чего попробуем генерировать новый текст.

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from collections import Counter

In [ ]:
# фиксируем воспроизводимость
def seed_everything(seed=42):

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=7575)

## Загрузим данные


In [ ]:
# open text file and read in data as `text`
with open("/content/Uolles_D._Velikieromanyi._Beskonechnaya_Shutka.txt", "r") as f:
    text = f.read()

Посмотрим первые 100 символов:

In [ ]:
text[:100]

'Бесконечная шутка\nДэвид Фостер Уоллес\n\n\nВеликие романы\nВ недалеком будущем пациенты реабилитационной'

### Токенизация

В ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [ ]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Посмотрим как символы закодировались целыми числами

In [ ]:
encoded[:100]

array([143,  90, 112, 137, 146,  29,  90,   0,  29, 115, 103,  72, 154,
        49, 129, 137, 115, 127, 140,  78,  36,  23, 152,  72, 107, 146,
       112, 129,  90, 156,  72,  44, 146,   5,   5,  90, 112, 127, 127,
       127,  76,  90,   5,  23, 137,  23,  90,  72, 156, 146,  25, 115,
        29,  99, 127,  76,  72,  29,  90, 152, 115,   5,  90, 137, 146,
        25,  72,  17,  49, 152,  49,  27,  90,  25,  72,  94, 115, 145,
        23,  90,  29, 129,  99,  72, 156,  90, 115,  17,  23,   5,  23,
       129, 115, 145,  23, 146,  29,  29, 146, 170])

## Предобработка данных



In [ ]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.0

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [ ]:
def get_batches(int_words, batch_size, seq_length):
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("Training on GPU!")
else:
    print("No GPU available, training on CPU; consider making n_epochs very small.")

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True
        )

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        """Forward pass through the network.
        These inputs are x, and the hidden/cell state `hidden`."""

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
            )

        return hidden

## Обучим модель



In [ ]:
def train(
    net,
    data,
    epochs=10,
    batch_size=10,
    seq_length=50,
    lr=0.001,
    clip=5,
    val_frac=0.1,
    print_every=10,
):
    """Training a network

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    """
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if train_on_gpu:
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size * seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if train_on_gpu:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(
                        output, targets.view(batch_size * seq_length).long()
                    )

                    val_losses.append(val_loss.item())

                net.train()  # reset to train mode after iterationg through validation data

                print(
                    "Epoch: {}/{}...".format(e + 1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)),
                )

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [ ]:
# define and print the net
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(172, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=172, bias=True)
)


### Установим гиперпараметры

In [ ]:
batch_size = 128
seq_length = 200
n_epochs = 50

# train the model
train(
    net,
    encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=0.001,
    print_every=10,
)

Epoch: 1/50... Step: 10... Loss: 3.4749... Val Loss: 3.5546
Epoch: 1/50... Step: 20... Loss: 3.4495... Val Loss: 3.4744
Epoch: 1/50... Step: 30... Loss: 3.3736... Val Loss: 3.4520
Epoch: 1/50... Step: 40... Loss: 3.3488... Val Loss: 3.4415
Epoch: 1/50... Step: 50... Loss: 3.3693... Val Loss: 3.4433
Epoch: 1/50... Step: 60... Loss: 3.3651... Val Loss: 3.4376
Epoch: 1/50... Step: 70... Loss: 3.3331... Val Loss: 3.4343
Epoch: 1/50... Step: 80... Loss: 3.3756... Val Loss: 3.4207
Epoch: 1/50... Step: 90... Loss: 3.3133... Val Loss: 3.4223
Epoch: 1/50... Step: 100... Loss: 3.3390... Val Loss: 3.4129
Epoch: 1/50... Step: 110... Loss: 3.3061... Val Loss: 3.4019
Epoch: 2/50... Step: 120... Loss: 3.2897... Val Loss: 3.3898
Epoch: 2/50... Step: 130... Loss: 3.2949... Val Loss: 3.3456
Epoch: 2/50... Step: 140... Loss: 3.2434... Val Loss: 3.2764
Epoch: 2/50... Step: 150... Loss: 3.1525... Val Loss: 3.1925
Epoch: 2/50... Step: 160... Loss: 3.0471... Val Loss: 3.0922
Epoch: 2/50... Step: 170... Loss:

## Checkpoint



In [ ]:
# change the name, for saving multiple files
model_name = "char_lstm_50_epoch.net"

# checkpoint = {
#     "n_hidden": net.n_hidden,
#     "n_layers": net.n_layers,
#     "state_dict": net.state_dict(),
#     "tokens": net.chars,
# }

# with open(model_name, "wb") as f:
#     torch.save(checkpoint, f)

torch.save(net, model_name)

---
## Делаем предсказания



In [ ]:
def predict(net, char, h=None, top_k=None):
    """Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    """

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_gpu:
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if train_on_gpu:
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [ ]:
def sample(net, size, prime="Инканденца", top_k=None):

    if train_on_gpu:
        net.cuda()
    else:
        net.cpu()

    net.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [ ]:
print(sample(net, 1000, prime="Орин Инканденца", top_k=2))

Орин Инканденца выстреивает с поднятой собственной спинкой в положении и со своими поднятом столовых сторон и после простора в полу при столовом подобной противорезивов с поднятыми половенками и приверживыми состояниями положения сталового подставка и стало пробежеть, и подозревали, когда она, при этом просто происходить, как он не просто принимает при этом по середине собственного поднятая соседняя сторона старой проблемы в подобных полевениях в положении, как признаки собравая положительное стекло, когда просто после подняли столько под подбородком и проблемам в картридже в присутствии из-за того, что под столом поднял под просто под простой приставной стекло приступи в подскочке и проблема при себе в получение привычка в получерия по себе в просторном состоянии, пока не собирается, когда под подном стальной подозрения в получении проблемах и старших стенах, и просто поднимает под собой, пока не поднимается на себя в полу под простой полотенце и поднял себе в компании и после простор

## Loading a checkpoint

Цитата из текста: 'Талант сам по себе ожидание... либо оправдываешь его, либо он машет платочком, исчезая навсегда.'

In [ ]:
loaded = torch.load(model_name)

In [ ]:
# Sample using a loaded model
print(sample(loaded, 1000, top_k=5, prime="Талант сам по себе ожидание"))

Талант сам по себе ожидание, никто не заметил. Последний стекленная концепта своего вопроса просмотра стоика из них, пока все все это со всей предметы просто переживающего сексуальное кориченио и продвигаться по картреджам на коленях и сообщает высокий корт.

– Посмотреть про себя и стало себе, и ты воспринимаешь.

– Но не ступает на комнате по поведении выражения.

– Не обремо просто не знаешь, то еще приставить, наверное, не сказал он, вот так и вы пришлось.

– Призрак?

– Я ты дейся и никогда, в стене, – она показывает, что все еще не помогла на проблему историю при этом образи, когда показал столок, – совершенно высокая последний матчей на паре, который всегда все стоял на семейку на секундой в своей стороне классического передавления.

– Только по твоему тень, когда она сказала:

– Я поднимаюсь в трезвом степени. Но не опускался стен и поднимающийся в круглие, чтобы он последние сложные представьем корте, когда в том, что ты на востоке со всей виней. Он не выбирают не столько, но 

# Word-Level LSTM

## Токенизация

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter

import numpy as np

In [ ]:
text[:100]

'Бесконечная шутка\nДэвид Фостер Уоллес\n\n\nВеликие романы\nВ недалеком будущем пациенты реабилитационной'

In [ ]:
# Токенизируем слова
# Разделяем текст на
words = text.split()
word_counts = Counter(words)
vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(len(words)-SEQUENCE_LENGTH)]
print(len(vocab))
print(len(word_to_int))
print(int_to_word[2])

117706
117706
Дэвид


## Dataloaders

In [ ]:
class TextDataset(Dataset):
    def __init__(self, samples, word_to_int):
        self.samples = samples
        self.word_to_int = word_to_int
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.word_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.word_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

In [ ]:
BATCH_SIZE = 32
dataset = TextDataset(samples, word_to_int)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)
print(dataset[1])

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 14, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 14, 56, 57, 58, 59, 60, 61, 62]), tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 14, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
        55, 14, 56, 57, 58, 59, 60, 61, 62, 63]))


In [ ]:
class TextGenerationLSTM(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_size,
        num_layers
    ):
        super(TextGenerationLSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        # nn.init.xavier_uniform_(self.lstm.weight)

        self.fc = nn.Linear(hidden_size, vocab_size)

        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0])
        x = self.embedding(x)
        out, (h_n, c_n) = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)

        return out, (h_n, c_n)


    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return h0, c0

## Установим гиперпараметры

In [ ]:
# Training Setup
embedding_dim = 32
hidden_size = 64
num_layers = 1
learning_rate = 0.01
epochs = 6

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextGenerationLSTM(
    vocab_size,
    embedding_dim,
    hidden_size,
    num_layers
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Обучим модель

In [ ]:
# Training
def train(model, epochs, dataloader, criterion):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs, _ = model(input_seq)
            loss = criterion(outputs, target_seq.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch} loss: {epoch_loss:.3f}")
train(model, epochs, dataloader, criterion)

Epoch 0 loss: 2.465
Epoch 1 loss: 1.355


In [ ]:
print(model)

## Делаем предсказания

In [ ]:
model_name = 'word_lstm_5_epoch.net'
torch.save(model, model_name)

In [ ]:
loaded_word = torch.load(model_name)

In [ ]:
def generate_text(model, start_string, num_words):
    model.eval()
    words = start_string.split()
    for _ in range(num_words):
        input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0).to(device)
        h, c = model.init_hidden(1)
        output, (h, c) = model(input_seq, (h, c))
        next_token = output.argmax(1)[-1].item()
        words.append(int_to_word[next_token])
    return " ".join(words)

# Example usage:
print("Generated Text:", generate_text(loaded_word, start_string="Талант сам по себе", num_words=1000))

**Выводы:**

- Построены две модели разных подходов: посимвольной генерации и пословной.
- Метрики моделей достаточно близки
- Конечный результат далёк от идеального. Возможно это из-за выбранного текста: объёмный + большое количество слов придумано автором + в данном случае видим мы считываем перевод изначально достаточно сложного текста, который принципиально непереводим. Так что возможно это повлияло на результат. Либо возможно следовало усложнить модели.